In [48]:
import pandas as pd

In [49]:
from aicentro.session import Session
session = Session(verify=False)

In [50]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact

from sklearn import preprocessing
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [51]:
nvoc_df = pd.read_csv('/aihub/data/nvocdata_final_20200720.csv', encoding = 'UTF-8')

In [52]:
del nvoc_df['Unnamed: 0']

In [53]:
nvoc_df.corr()

,상담대분류,상담소분류,상품구분,영업단,급지,AS신청유형,AS원인유형,AS처리유형,cnslt_rscs_diff,as_rscs_diff,rscs_yn
상담대분류,1.000000,0.069312,-1.000000,0.028334,-0.093510,-0.026543,0.153577,0.027245,0.150649,0.129595,-0.200736
상담소분류,0.069312,1.000000,-0.069312,-0.055562,-0.057038,-0.047027,0.004922,-0.030815,-0.058146,-0.073440,0.021220
상품구분,-1.000000,-0.069312,1.000000,-0.028334,0.093510,0.026543,-0.153577,-0.027245,-0.150649,-0.129595,0.200736
영업단,0.028334,-0.055562,-0.028334,1.000000,-0.297507,-0.077719,0.092817,0.158349,-0.010757,0.026178,-0.020036
급지,-0.093510,-0.057038,0.093510,-0.297507,1.000000,0.022677,-0.050340,-0.014775,-0.068673,0.016906,-0.047739
AS신청유형,-0.026543,-0.047027,0.026543,-0.077719,0.022677,1.000000,0.003673,-0.006981,0.021067,-0.013123,0.009264
AS원인유형,0.153577,0.004922,-0.153577,0.092817,-0.050340,0.003673,1.000000,0.839792,0.025142,0.076443,-0.008181
AS처리유형,0.027245,-0.030815,-0.027245,0.158349,-0.014775,-0.006981,0.839792,1.000000,-0.009841,0.072479,-0.018618
cnslt_rscs_diff,0.150649,-0.058146,-0.150649,-0.010757,-0.068673,0.021067,0.025142,-0.009841,1.000000,0.504470,-0.619433
as_rscs_diff,0.129595,-0.073440,-0.129595,0.026178,0.016906,-0.013123,0.076443,0.072479,0.504470,1.000000,-0.341389


In [54]:
nvoc_df

,상담대분류,상담소분류,상담일자,상품구분,영업단,급지,해지일,AS신청유형,AS원인유형,AS처리유형,AS완료일자,AS처리시간,cnslt_rscs_diff,as_rscs_diff,rscs_yn
0,1,9,2016-01-08,0,1,4,2016-10-14,15,62,57,2016-01-08,28,280,280.0,False
1,1,9,2016-01-09,0,0,2,2017-04-17,25,62,57,2016-01-09,24,464,464.0,False
2,1,9,2016-01-09,0,0,2,2017-04-17,25,86,40,2014-02-18,24,464,1154.0,False
3,1,9,2016-01-09,0,0,2,2017-04-17,25,0,1,2016-01-08,1,464,465.0,False
4,1,9,2016-01-09,0,0,2,2017-04-17,13,62,57,2015-11-30,25,464,504.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117,1,0,2020-06-10,0,1,2,2020-06-15,25,81,76,2018-08-31,2,5,654.0,True
2118,1,0,2020-06-10,0,1,2,2020-06-15,23,49,0,2016-07-29,39,5,1417.0,True
2119,1,0,2020-06-10,0,1,2,2020-06-15,18,62,57,2017-11-01,0,5,957.0,True
2120,1,0,2020-06-10,0,1,2,2020-06-15,15,62,57,2015-11-12,0,5,1677.0,True


In [55]:
# 사용할 X컬럼들과 Y컬럼을 지정한다
x_columns = ['상담대분류', '상담소분류', '급지', 'AS신청유형', 'AS원인유형', 'AS처리유형']
y_column  = ['rscs_yn']

In [56]:
# # 데이터를 읽고 순서를 섞는다
nvoc_model_df=nvoc_df.sample(frac=1).reset_index(drop=True)
nvoc_model_df

,상담대분류,상담소분류,상담일자,상품구분,영업단,급지,해지일,AS신청유형,AS원인유형,AS처리유형,AS완료일자,AS처리시간,cnslt_rscs_diff,as_rscs_diff,rscs_yn
0,1,6,2017-11-09,0,2,2,2019-06-17,11,85,84,2017-02-13,7,585,854.0,False
1,1,4,2017-03-16,0,0,1,2020-02-13,26,86,40,2016-05-18,46,1064,1366.0,False
2,1,2,2017-03-15,0,0,1,2019-03-05,15,57,48,2017-03-15,3,720,720.0,False
3,1,4,2017-06-22,0,2,3,2019-10-01,23,62,57,2017-05-20,1,831,864.0,False
4,1,6,2017-03-16,0,2,0,2020-06-05,13,62,50,2017-11-03,4,1177,945.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117,1,11,2016-08-18,0,0,5,2019-06-12,15,53,46,2016-09-06,67,1028,1009.0,False
2118,1,8,2018-04-06,0,1,3,2018-08-27,13,85,84,2017-09-28,23,143,333.0,False
2119,1,5,2016-05-13,0,0,2,2017-06-20,15,62,57,2017-04-29,12,403,52.0,False
2120,1,0,2016-11-29,0,2,3,2016-12-14,20,62,57,2014-10-16,4,15,790.0,True


In [57]:
# Y컬럼의 문자열값을 LabelEncoder를 통해 숫자로 바꾼다.
y_true_label_encoder = preprocessing.LabelEncoder()
y_true_encoded       = y_true_label_encoder.fit_transform(nvoc_model_df[y_column])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [63]:
y_true_encoded

array([0, 0, 0, ..., 0, 1, 0])

In [64]:
k=9
model = KMeans(n_clusters=k, random_state=0, n_init=100)  # k개 클러스터로 모델 생성
cluster_labels = model.fit_predict(nvoc_model_df[x_columns]) # X컬럼으로 지정된 필드값으로 피팅
y_pred_label_encoder = preprocessing.LabelEncoder() # 예측한 클러스터에서 사용할 라벨인코더
y_pred_label_encoder.fit(np.array(['unknown ' + str(i+1) for i in range(0, k, 1)], dtype=object))

for pred_label_num in range(0,k,1): # 각 클러스터 순회        
    # 해당 클러스터에서 가장 많이 출력한 실제 값의 인덱스를 구한다
    most_frequent_index = np.argmax(np.bincount(y_true_encoded[cluster_labels==pred_label_num]))
    print(f'{most_frequent_index}')
    # 구한 인덱스의 라벨을 구해서 해당클러스터 라벨로 업데이트한다
    y_pred_label_encoder.classes_[pred_label_num] = y_true_label_encoder.classes_[most_frequent_index]

0
0
0
0
0
0
0
0
0


In [65]:
model.inertia_

119606.11348144148

In [66]:
silhouette_score(nvoc_model_df[x_columns].values, cluster_labels)

0.3930154265433816

In [67]:
# Y컬럼의 문자열값을 LabelEncoder를 통해 숫자로 바꾼다.
y_true_label_encoder = preprocessing.LabelEncoder()
y_true_encoded       = y_true_label_encoder.fit_transform(nvoc_model_df[y_column])

kMeansModels              = dict() # k값별 모델 저장할 딕셔너리
kMeansModelPreds          = dict() # k값별 모델 예측 결과 저장할 딕셔너리
kMeansModelLabelEncoder   = dict() # k값별 라벨인코더 저장할 딕셔너리

sumSquaredDistancesList   = list() # 샘플과 클러스터 센터간 거리 제곱의 합 리스트
silhouetteScoreList       = list() # Silhouette Coefficient 평균 리스트

ks = [2,3,4,5,6,7,8,9] # k값으로 2부터 9까지 테스트한다

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [68]:
y_pred_label_encoder = preprocessing.LabelEncoder() # 예측한 클러스터에서 사용할 라벨인코더
# 초기 임의 값 (unknown 1, unknown 2...) 으로 인코딩한다
y_pred_label_encoder.fit(np.array(['unknown ' + str(i+1) for i in range(0, k, 1)], dtype=object))
for pred_label_num in range(0,k): # 각 클러스터 순회        
    # 해당 클러스터에서 가장 많이 출력한 실제 값의 인덱스를 구한다
    most_frequent_index = np.argmax(np.bincount(y_true_encoded[cluster_labels==pred_label_num]))
    # 구한 인덱스의 라벨을 구해서 해당클러스터 라벨로 업데이트한다
    y_pred_label_encoder.classes_[pred_label_num] = y_true_label_encoder.classes_[most_frequent_index]

kMeansModels[k]     = model                       # 모델 저장
kMeansModelPreds[k] = cluster_labels              # 모델 예측결과 저장     
kMeansModelLabelEncoder[k] = y_pred_label_encoder # 라벨인코더 저장
sumSquaredDistancesList.append(model.inertia_)    # # 샘플과 클러스터 센터간 거리 제곱의 합 저장
silhouetteScoreList.append(silhouette_score(nvoc_model_df[x_columns].values, cluster_labels)) # Silhouette Score저장

In [69]:
for k in ks: # k값 순회
    model = KMeans(n_clusters=k, random_state=0, n_init=100)  # k개 클러스터로 모델 생성
    cluster_labels = model.fit_predict(nvoc_model_df[x_columns]) # X컬럼으로 지정된 필드값으로 피팅
    
    y_pred_label_encoder = preprocessing.LabelEncoder() # 예측한 클러스터에서 사용할 라벨인코더
    # 초기 임의 값 (unknown 1, unknown 2...) 으로 인코딩한다
    y_pred_label_encoder.fit(np.array(['unknown ' + str(i+1) for i in range(0, k, 1)], dtype=object))
    for pred_label_num in range(0,k,1): # 각 클러스터 순회        
        # 해당 클러스터에서 가장 많이 출력한 실제 값의 인덱스를 구한다
        most_frequent_index = np.argmax(np.bincount(y_true_encoded[cluster_labels==pred_label_num]))
        # 구한 인덱스의 라벨을 구해서 해당클러스터 라벨로 업데이트한다
        y_pred_label_encoder.classes_[pred_label_num] = y_true_label_encoder.classes_[most_frequent_index]
    
    kMeansModels[k]     = model                       # 모델 저장
    kMeansModelPreds[k] = cluster_labels              # 모델 예측결과 저장     
    kMeansModelLabelEncoder[k] = y_pred_label_encoder # 라벨인코더 저장
    sumSquaredDistancesList.append(model.inertia_)    # # 샘플과 클러스터 센터간 거리 제곱의 합 저장
    silhouetteScoreList.append(silhouette_score(nvoc_model_df[x_columns].values, cluster_labels)) # Silhouette Score저장

In [72]:
def render_plot(x_col='petal_length', y_col='petal_width', k=3):
    y_pred        = kMeansModelPreds[k]        # 모델 예측값
    label_encoder = kMeansModelLabelEncoder[k] # 라벨인코더
    
    # 원본과 예측값을 합쳐 데이터셋을 준비
    mdf = pd.concat([nvoc_model_df, pd.DataFrame(label_encoder.inverse_transform(y_pred), columns=[y_column+'_pred'])], axis=1)
    
    plt.figure(figsize=(15,8)) # 출력 크기를 지정
    
    ddf = mdf[mdf[y_column] != mdf[y_column+'_pred']] # 실제라벨과 예측라벨이 틀린 경우 추출
    # X기호로 error임을 표시한다
    plt.scatter(x=ddf[x_col], y=ddf[y_col], c='black', marker='x', s=300, label='error') 
    
    colors = ['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black'] # 컬러값 리스트
    for (idx,cls) in enumerate(list(y_true_label_encoder.classes_)):
        ddf = mdf[mdf[y_column] == cls] # 실제라벨 데이터는 작은 점으로 출력
        plt.scatter(x=ddf[x_col], y=ddf[y_col], c=colors[idx], marker='.', linewidths=3, s=50, label=cls)
        ddf = mdf[mdf[y_column+'_pred'] == cls] # 예측라벨 데이터는 큰 원으로 출력
        plt.scatter(x=ddf[x_col], y=ddf[y_col], c=colors[idx], marker='o', linewidths=13, s=2, label=cls+' pred')
    
    plt.legend();plt.xlabel(x_col);plt.ylabel(y_col)
    print('accuracy {:.2f}'.format(np.mean(mdf[y_column] == mdf[y_column+'_pred']))) # 정확도 출력
    plt.show()
    

In [73]:
interact(render_plot, x_col=x_columns, y_col=x_columns, k=ks)

interactive(children=(Dropdown(description='x_col', options=('상담대분류', '상담소분류', '급지', 'AS신청유형', 'AS원인유형', 'AS처리…

<function __main__.render_plot(x_col='petal_length', y_col='petal_width', k=3)>

In [ ]:
model.cluster_centers_